<h1>BATTLE OF THE NEIGHBORHOODS

In [1]:
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import requests # Library for website scraping

!pip install BeautifulSoup4
from bs4 import BeautifulSoup

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

from sklearn.cluster import KMeans # import k-means from clustering stage

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

     |████████████████████████████████| 122kB 5.8MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following package

In [2]:
#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, "html.parser")

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df = canada_df.groupby(["Postal Code", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [3]:
print("Shape: ", canada_df.shape)

Shape:  (103, 3)


In [4]:
df_geo_coor = pd.read_csv("Geospatial_Coordinates.csv")
df_toronto = pd.merge(canada_df, df_geo_coor, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [5]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="'toronto'_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [6]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [7]:
df_toronto_TT = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_TT.head()

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [8]:
map_toronto_TT = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(
        df_toronto_TT['Latitude'], 
        df_toronto_TT['Longitude'], 
        df_toronto_TT['Borough'], 
        df_toronto_TT['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_TT)  

map_toronto_TT

In [9]:
neighborhood_name = df_toronto_TT.loc[0, 'Neighborhood']
neighborhood_latitude = df_toronto_TT.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_TT.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [10]:
CLIENT_ID = 'U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH' # your Foursquare ID
CLIENT_SECRET = 'HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH
CLIENT_SECRET:HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG


In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH&client_secret=HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG&v=20180604&ll=43.67635739999999,-79.2930312&radius=500&limit=100'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
nearby_venues_TT = getNearbyVenues(names=df_toronto_TT['Neighborhood'],
                                   latitudes=df_toronto_TT['Latitude'],
                                   longitudes=df_toronto_TT['Longitude']
                                  )
nearby_venues_TT.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [16]:
nearby_venues_TT.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [17]:
#Unique categories from all venues
print('There are {} uniques categories.'.format(len(nearby_venues_TT['Venue Category'].unique())))

There are 238 uniques categories.


In [18]:
# one hot encoding
toronto_TT_onehot = pd.get_dummies(nearby_venues_TT[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_TT_onehot['Neighborhood'] = nearby_venues_TT['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_TT_onehot.columns[-1]] + list(toronto_TT_onehot.columns[:-1])
toronto_TT_onehot = toronto_TT_onehot[fixed_columns]

toronto_TT_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_TT_grouped = toronto_TT_onehot.groupby('Neighborhood').mean().reset_index()
toronto_TT_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.041667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.055556,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,Central Bay Street,0.015625,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.015625,0.0,0.0,0.015625,0.0,0.0,0.0


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_TT_grouped['Neighborhood']

for ind in np.arange(toronto_TT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_TT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Eastern European Restaurant,Department Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Gym,Grocery Store,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Bar,Harbor / Marina,Coffee Shop,Boat or Ferry,Rental Car Location,Boutique,Plane
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Japanese Restaurant,Department Store,Salad Place,Bubble Tea Shop,Yoga Studio


In [21]:
num_top_venues = 10

for hood in toronto_TT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_TT_grouped[toronto_TT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.04
3         Cheese Shop  0.04
4              Bakery  0.04
5                Café  0.04
6            Beer Bar  0.04
7  Seafood Restaurant  0.04
8       Shopping Mall  0.02
9      Breakfast Spot  0.02


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.12
1                  Bakery  0.08
2          Breakfast Spot  0.08
3             Coffee Shop  0.08
4             Yoga Studio  0.04
5           Burrito Place  0.04
6            Climbing Gym  0.04
7       Convenience Store  0.04
8  Furniture / Home Store  0.04
9           Grocery Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1           Yoga Studio  0.06
2         Auto Workshop  0.06
3  Gym / Fitness Center  0.06
4         Garden Cen

In [22]:
# set number of clusters
kclusters = 5

toronto_TT_grouped_clustering = toronto_TT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_TT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_TT_merged = df_toronto_TT

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_TT_merged = toronto_TT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_TT_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Women's Store
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Bookstore,Ice Cream Shop,Caribbean Restaurant,Pub,Lounge
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,Park,Fast Food Restaurant,Gym,Ice Cream Shop,Fish & Chips Shop,Steakhouse,Restaurant,Pub,Italian Restaurant,Burrito Place
3,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Latin American Restaurant,Bar,Ice Cream Shop,Fish Market
4,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [24]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_TT_merged['Latitude'], 
        toronto_TT_merged['Longitude'], 
        toronto_TT_merged['Neighborhood'], 
        toronto_TT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

OK SO FOR EXAMPLE I AM A TOURIST WHO IS VERY INTERESTED IN STUDIO DISTRICT OF EAST TORONTO AND SINCE THE MOST POPULAR VENUE IS café, I SHALL HEAD THERE!

In [25]:
search_query = 'café'
radius = 500
Limit = 100
print(search_query + ' .... OK!')

café .... OK!


In [26]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH&client_secret=HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG&ll=43.659526,-79.340923&v=20180604&query=Café&radius=500&Limit=100'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, Limit)
url

'https://api.foursquare.com/v2/venues/search?client_id=U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH&client_secret=HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG&ll=43.659526,-79.340923&v=20180604&query=Café&radius=500&Limit=100'

In [27]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Purple Penguin Cafe,Café,889 Queen St East,Logan Ave,43.660501,-79.342565,"[{'label': 'display', 'lat': 43.66050115073973...",171,M4M 1J4,CA,Toronto,ON,Canada,"[889 Queen St East (Logan Ave), Toronto ON M4M...",5475df76498eab68d4a3a6b9
1,Cafe Florentin II,Bakery,948 Queen ST E,Carlaw AV,43.660738,-79.341278,"[{'label': 'display', 'lat': 43.66073792003301...",137,NaN,CA,Toronto,ON,Canada,"[948 Queen ST E (Carlaw AV), Toronto ON, Canada]",4c9ca89d7c096dcbe75ac3d1
2,River Rock Cafe,Café,NaN,NaN,43.661416,-79.340155,"[{'label': 'display', 'lat': 43.661416, 'lng':...",219,NaN,CA,NaN,NaN,Canada,[Canada],5102d80be4b04ee89d6da2bc
3,Underground Cafe,Smoke Shop,NaN,NaN,43.659223,-79.344804,"[{'label': 'display', 'lat': 43.65922284251243...",314,NaN,CA,NaN,NaN,Canada,[Canada],521bb904498ee10638f3fac2
4,RiverRock Cafe,Café,181A Carlaw Ave,Queen St. East,43.661497,-79.340235,"[{'label': 'display', 'lat': 43.66149725141493...",226,M4M 2S1,CA,Toronto,ON,Canada,"[181A Carlaw Ave (Queen St. East), Toronto ON ...",4c755c282db5236a4dc8bc79
5,Pic Nic at Home Bakery & Cafe,Bakery,753 Queen Street E,NaN,43.658579,-79.347092,"[{'label': 'display', 'lat': 43.65857860513276...",507,NaN,CA,Toronto,ON,Canada,"[753 Queen Street E, Toronto ON, Canada]",50ad62d3e4b0602cfa62ab67


OKAY SO PURPLE PENGUIN CAFE SOUNDS REALLY INTERESTING AND UNIQUE! SO LET'S GO! BUT WAIT A MINUTE, BEFORE THAT, LET'S CHECK OUT THE LOCATION FIRST USING THE CRIME DATA.

In [28]:
## Purple Penguin Cafe
venue_id = '5475df76498eab68d4a3a6b9'
LIMIT = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/5475df76498eab68d4a3a6b9/tips?client_id=U4XOIF34P5OIL1OBDEFTICMN1AFGJ3LHFL1N1RCSQGI2UXAH&client_secret=HKJIEFPIUKXZ0I2ZS3SS104Q0KCEI5TIJM1S1UUQNUZ5HVRG&v=20180604&limit=15'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee214e55b1a341f381c0515'},
 'response': {'tips': {'count': 8,
   'items': [{'id': '582d2cfcb4ddb3469e3a6b07',
     'createdAt': 1479355644,
     'text': 'The owner is a sweetheart. They serve GenuineTea matcha here imported from Japan! It is creamy and delicious. I had the green eggs and ham (spinach, boiled eggs, and ham on a multi-grain croissant).',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/582d2cfcb4ddb3469e3a6b07',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 1,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '13658912',
      'firstName': 'Emilie',
      'lastName': 'W',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/13658912-Z34EGHKSQPQ5SIQH.jpg'}},
     'authorInteractionType': 'liked'}]}}}